# Filter PSDs for wind speed

In [ ]:
from pathlib import Path
import numpy as np

from obspy.core import UTCDateTime as UTC

from data_quality_control import analysis, dqclogging, timelist 

# Only for display in documentation!
from IPython.core.display import display, HTML 

## Input parameters

In [ ]:
starttime = UTC("2020-12-20")
endtime = UTC("2021-01-12")
#starttime = UTC("2020-01-01")
#endtime = UTC("2021-12-31")

In [ ]:
# NSLC
nslc_code = "GR.BFO..BHZ"
datadir = Path('../sample_output/show_processing/')

winddatafile = Path("../testing/winddata_bfo.txt")

## Set logger

In [ ]:
logfilename = datadir.joinpath("read_winddata.log")
dqclogging.configure_handlers(analysis.logger, "INFO", "DEBUG", logfilename)

## Get infos on data availability

In [ ]:
lyza = analysis.Analyzer(datadir, nslc_code,
                            fileunit="year")

In [ ]:
print(lyza)

In [ ]:
files = lyza.get_available_datafiles()
print(files)

In [ ]:
sdate, edate = lyza.get_available_timerange()
print(sdate, edate)

## Get wind data

### Read wind data

First, we read the raw wind speed data from the text file.

In [ ]:
#reload(timelist)
awind = timelist.read_winddata(winddatafile)

###  Interpolate wind data

Wind speed is available about every 6 hours. The power spectral densities are available
every 1 hour. We want to filter the PSDs for times when wind speed is in a specific range.
To do so, we interpolate the wind speed to the finer grid.

In [ ]:
timedelta_wind = np.ediff1d(awind[:,0])
print("Report interval of wind speed is approx. {:.0f} h".format(np.mean(timedelta_wind) / 3600))

In [ ]:
x, f = timelist.read_interp_winddata(winddatafile, starttime, endtime, 3600)

In [ ]:
timedelta_wind = np.ediff1d(x)
print("New interval of wind speed is {:.1f} h".format(np.mean(timedelta_wind) / 3600))

## Filter wind speed

From the regularly sampled wind data, we can extract those times 
when windspeed is e.g. $3 \leq c \leq 6$ m/s.

In [ ]:
# Extract times with certain wind speed
cmin = 3
cmax = 6
xsel = x[np.logical_and(f >= cmin, f <= cmax)]
tlist = [UTC(xi) for xi in xsel]
print("For {:d} samples of {} is wind speed between {:g} and {:g} m/s".format(
    len(tlist), len(x), cmin, cmax))

In [ ]:
DATA = lyza.get_data(tlist)
print(lyza.starttime, lyza.endtime)
print(lyza.amps.shape)
print("PSD shape:", lyza.psds.shape)
print("len(tlist):", len(tlist))

fig = lyza.plot_spectrogram()#cmap=plt.cm.gist_heat, vmin=-0.7, vmax=2.4)
#fig_cont.savefig(datadir.joinpath("spectrogram_wind_{:g}-{:g}mps.png"))

In [ ]:
fig_psd = lyza.plot3d_psds()
display(HTML(fig_psd.to_html(include_mathjax="cdn")))

Note, that the plotting currently uses the time list as axis. Therefore pixels are not equally wide but
stretched/squeezed until the next timestamp.

## Continuous time range

For comparison, let's see how the spectrogram looks like when
we use the full available time range.

In [ ]:
DATA = lyza.get_data(starttime, endtime)

In [ ]:
print(DATA)

In [ ]:
fig = lyza.plot_spectrogram()

In [ ]:
fig_psd = lyza.plot3d_psds()
display(HTML(fig_psd.to_html(include_mathjax="cdn")))